### EXAMPLE 1

In [7]:
# Keep in Mind - A LightningModule is a PyTorch nn.Module - it just has a few more helpful features.
## training_step과 configure_optimizers 필수적으로 구현해야 합니다.

import os

import torch
from pytorch_lightning import LightningModule, Trainer
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms
from torchvision.datasets import MNIST

AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")


class MNISTModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = torch.nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

## training step에서는 꼭 log뿐만 아니라 return loss를 해줘야한다
# self.log하는거는 매번 다 logging되고 이를 callback에서 활용할 수 있다. 
## validation step에서는 할 필요 없지만. 

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        # return loss
        self.log('chkimsu_loss',loss * 2, prog_bar = True, on_step= True)
        return loss 

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

In [16]:
# Init our model
mnist_model = MNISTModel()
import pytorch_lightning

checkpoint_callback = pytorch_lightning.callbacks.ModelCheckpoint('20220427',
                                                  save_top_k=2, monitor='chkimsu_loss', mode='min', save_last = True)


# Init DataLoader from MNIST Dataset
train_ds = MNIST(PATH_DATASETS, train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

# Initialize a trainer
trainer = Trainer(
    gpus=AVAIL_GPUS,
    max_epochs=10,
    progress_bar_refresh_rate=20,
    checkpoint_callback = checkpoint_callback
)
    
# Train the model ⚡
trainer.fit(mnist_model, train_loader)

C:\Users\chkim\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x0000023D1C093848>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x0000023D1C093848>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 7.9 K 
--------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 938/938 [00:09<00:00, 95.80it/s, loss=0.683, v_num=17, chkimsu_loss=0.795] 


In [5]:
!tensorboard --logdir lightning_logs/

In [ ]:
class Classifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28 * 28, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 10)
        )


    #   forward는 모델의 추론 결과를 제공하고 싶을 때 사용합니다. 
    #   nn.Module처럼 꼭 정의해야 하는 메서드는 아니지만 self(<입력>)과 같이 사용할 수 있게 만들어주므로 구현해주면 다른 메서드를 구현할 때 편리합니다.
    
    def forward(self, x):
        return self.model(x)


    # training_step은 학습 루프의 body 부분을 나타냅니다. 
    # 이 메소드에서는 argument로 training 데이터로더가 제공하는 batch와 해당 batch의 인덱스가 주어지고 학습 로스를 계산하여 리턴합니다.
    #  pytorch lightning은 편리하게도 batch의 텐서를 cpu 혹은 gpu 텐서로 변경하는 코드를 따로 추가하지 않아도 trainer의 설정에 따라 자동으로 적절한 타입으로 변경해줍니다.

    def training_step(self, batch, batch_idx):

        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        return loss



#   validation_step은 학습 중간에 모델의 성능을 체크하는 용도로 사용합니다.
#    training_step과 마찬가지로 validation 데이터로더에서 제공하는 배치를 가지고 확인하고자 하는 통계량을 기록할 수 있습니다.
#     하나의 값을 저장할 때는 self.log(<변수 이름="">, <값>)과 같이 저장할 수 있고
#     여러 개의 변수를 저장하고 싶으면 아래 예시와 같이 self.log_dict로 변수 이름, 값 쌍을 가지고 있는 딕셔너리를 저장할 수 있습니다.
#      각 스탭마다 변수에 저장된 값의 평균이 해당 변수의 최종 값이 됩니다. 특별히 설정을 바꾸지 않으면 변수 중에 'val_loss'가 best 모델을 구하는 기준으로 사용됩니다.
## !!! 중요한 건 VALIDATION STEP의 마지막에 저장되는 값은 STEP마다 평균내진다는 것이ㅏㄷ. 

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        acc = FM.accuracy(logits, y)
        loss = F.cross_entropy(logits, y)
        metrics = {'val_acc': acc, 'val_loss': loss}
        self.log_dict(metrics)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        acc = FM.accuracy(logits, y)
        loss = F.cross_entropy(logits, y)
        metrics = {'test_acc': acc, 'test_loss': loss}
        self.log_dict(metrics)
    
    def configure_optimizers(self):
        optimizer = Adam(self.parameters())
        return optimizer

In [ ]:
pl.seed_everything(args.seed)

# dataloaders
dataset = MNIST('', train=True, download=True, transform=transforms.ToTensor())
train_dataset, val_dataset = random_split(dataset, [55000, 5000])
test_dataset = MNIST('', train=False, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=args.batch_size)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size)

In [ ]:
# 모델을 학습하기 위해서는 학습 로직을 정하는 Trainer를 생성해야 합니다. Pytorch lightning의 Trainer는 굉장히 많은 기능을 제공합니다. 
# 아래 예제에서는 간단히 학습 epoch 수와 gpu 수만 조정할 수 있도록 만들었습니다.

# gpus가 0일 때는 cpu를 사용하고 gpus가 1 이상이면 gpu를 사용하여 모델을 학습합니다.
# gpus가 2 이상이면 자동으로 다중 gpu를 활용해 분산 학습을 진행하게 되는데 기본 설정은 process를 spawn하는 distributed data parallel 방식(ddp_spawn)으로 되어있습니다.

# model
model = Classifier()

# training
trainer = pl.Trainer(max_epochs=args.n_epochs, gpus=args.n_gpus)
trainer.fit(model, train_loader, val_loader)




# Trainer의 test 함수에 test 데이터로더를 넘겨주어 모델을 테스트할 수 있습니다. 
# 아래와 같이 따로 어떤 모델을 테스트할지 지정하지 않으면 자동으로 Trainer가 validation을 통해 구한 best 모델을 가지고 테스트를 진행하게 됩니다.
trainer.test(test_dataloaders=test_loader)


## PL은 기본적으로 CHECKPOINT를 매 버전마다 저장해주지만 이 저장주기를 바꾸고 싶을때 CALLBACK을 이용하는 것!!!!


### EXAMPLE 2


In [6]:
import pytorch_lightning as pl
import torch
import torch.nn as nn

# Lightning Model 정의를 할 클래스에는 반드시 LightningModule을 상속받는다. (Like Torch's nn.Module)
# pretrained model을 생성하고 transfer learning을 위해 마지막 Linear layer의 출력을 1(for binary)로 바꿔준다.

from efficientnet_pytorch import EfficientNet
from sklearn.metrics import roc_auc_score

class Model(pl.LightningModule):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self.net = EfficientNet.from_pretrained(arch, advprop=True)
        self.net._fc = nn.Linear(in_features=self.net._fc.in_features, out_features=1, bias=True)

    def forward(self, x):  #model의 입력에 대한 output을 내는 forward
        return self.net(x)


    def configure_optimizers(self):  # 최적화를 위한 optimizer와 learning rate scheduler 초기화 및 반환
        optimizer = torch.optim.AdamW(self.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            max_lr=lr,
            epochs=max_epochs,
            optimizer=optimizer,
            steps_per_epoch=int(len(train_dataset) / batch_size),
            pct_start=0.1,
            div_factor=10,
            final_div_factor=100,
            base_momentum=0.90,
            max_momentum=0.95,
        )
        return [optimizer], [scheduler]


    def step(self, batch):  # forward and calculate loss
        # return batch loss
        x, y  = batch
        y_hat = self(x).flatten()
        y_smo = y.float() * (1 - label_smoothing) + 0.5 * label_smoothing
        loss  = F.binary_cross_entropy_with_logits(y_hat, y_smo.type_as(y_hat),
                                                   pos_weight=torch.tensor(pos_weight))
        return loss, y, y_hat.sigmoid()


    # 1 iteration에 대한 training
    # batch만큼의 output을 얻고 loss와 accuracy를 return
    
    def training_step(self, batch, batch_nb):
        # hardware agnostic training
        loss, y, y_hat = self.step(batch)
        acc = (y_hat.round() == y).float().mean().item()
        tensorboard_logs = {'train_loss': loss, 'acc': acc}
        return {'loss': loss, 'acc': acc, 'log': tensorboard_logs}



    # validation_step은 1 iteration에 대한 함수라고 하면  validation_epoch_end는 1 epoch에 대한 함수이다.

    # validation_step 함수의 역할은 training_step과 같은 역할을 하며 validation_epoch_end 함수는 logging과 학습과정에 대한 print를 위해 사용한다

    def validation_step(self, batch, batch_nb):
        loss, y, y_hat = self.step(batch)
        return {'val_loss': loss,
                'y': y.detach(), 'y_hat': y_hat.detach()}

    def validation_epoch_end(self, outputs):  # 한 에폭이 끝났을 때 실행
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        y = torch.cat([x['y'] for x in outputs])
        y_hat = torch.cat([x['y_hat'] for x in outputs])
        auc = AUROC()(y_hat, y) if y.float().mean() > 0 else 0.5 # skip sanity check
        acc = (y_hat.round() == y).float().mean().item()
        print(f"Epoch {self.current_epoch} acc:{acc} auc:{auc}")
        tensorboard_logs = {'val_loss': avg_loss, 'val_auc': auc, 'val_acc': acc}
        return {'avg_val_loss': avg_loss,
                'val_auc': auc, 'val_acc': acc,
                'log': tensorboard_logs}

    # 테스트이므로 정답이 없다. 
    def test_step(self, batch, batch_nb):
        x, _ = batch
        y_hat = self(x).flatten().sigmoid()
        return {'y_hat': y_hat}

    def test_epoch_end(self, outputs):
        y_hat = torch.cat([x['y_hat'] for x in outputs])
        df_test['target'] = y_hat.tolist()
        N = len(glob('submission*.csv'))
        df_test.target.to_csv(f'submission{N}.csv')
        return {'tta': N}


    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers,
                          drop_last=True, shuffle=True, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers,
                          drop_last=False, shuffle=False, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers,
                          drop_last=False, shuffle=False, pin_memory=False)

In [ ]:
checkpoint_callback = pl.callbacks.ModelCheckpoint('{epoch:02d}_{val_auc:.4f}',
                                                  save_top_k=1, monitor='val_auc', mode='max')
trainer = pl.Trainer(
    tpu_cores=tpu_cores,
    gpus=gpus,
    precision=16 if gpus else 32,
    max_epochs=max_epochs,
    num_sanity_val_steps=1 if debug else 0,  # catches any bugs in your validation without having to wait for the first validation check. 
    checkpoint_callback=checkpoint_callback
    )
    
trainer.fit(model)

### Pytorch 코드

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers,
                          drop_last=True, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers,
                          drop_last=False, shuffle=False, pin_memory=True)          
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers,
                          drop_last=False, shuffle=False, pin_memory=False)
                          
def train(epoch, model, optimizer, criterion, scaler, scheduler):
    model.train()
    train_loss = []
    correct = 0
    total = 0
    
    loop = tqdm(train_loader, desc=f'TRAIN-{epoch}', leave=True)
    for idx, (inputs, targets) in enumerate(loop):
        inputs, targets = inputs.to(device), targets.to(device)
        smooth_targets = targets.float() * (1 - label_smoothing) + 0.5 * label_smoothing

        with torch.cuda.amp.autocast():
            outputs = model(inputs).flatten()
            loss = criterion(outputs, smooth_targets)
        
        train_loss.append(loss.item())
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()


        output_enc = outputs.sigmoid().round()
        correct += (output_enc == targets).sum().item()
        total += targets.shape[0]

        mean_loss = sum(train_loss) / len(train_loss)
        mean_acc = correct / total * 100
        loop.set_postfix(loss=mean_loss, accuracy=mean_acc)
        
    scheduler.step(mean_loss) 
    
def valid(epoch, model, optimizer, criterion):
    model.eval()
    val_loss = []
    correct = 0
    total = 0
    ra = 0

    loop = tqdm(valid_loader, desc=f'VALID-{epoch}', leave=True)
    for idx, (inputs, targets) in enumerate(loop):
        inputs, targets = inputs.to(device), targets.to(device)
        
        with torch.no_grad():
            
            outputs = model(inputs).flatten() 
            loss = criterion(outputs, targets.float())
        

        val_loss.append(loss.item())
       
        output_enc = outputs.sigmoid().round()
        correct += (output_enc == targets).sum().item()
        total += targets.shape[0]

        ra += roc_auc_score(output_enc.cpu(),targets.cpu())
        
        mean_loss = sum(val_loss) / len(val_loss)
        mean_acc = correct / total * 100
        loop.set_postfix(loss=mean_loss, accuracy=mean_acc)

    print(f"Epoch {epoch} acc:{mean_acc} auc:{ra / len(train_loader)}")
    
def test(model, optimizer):
    model.eval()
    all = torch.tensor([])
    loop = tqdm(test_loader, desc=f'TEST', leave=True)
    for (inputs, _) in loop:
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(inputs).flatten().sigmoid()
        all = torch.cat((all, outputs))
        
    df_test['target'] = all.tolist()
    N = len(glob('submission*.csv'))
    df_test.target.to_csv(f'submission{N}.csv')
    return {'tta': N}
   
from efficientnet_pytorch import EfficientNet

def main():
    model = EfficientNet.from_pretrained(arch, advprop=True)
    model._fc = nn.Linear(model._fc.in_features, out_features=1, bias=True)
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss()
    scaler = torch.cuda.amp.GradScaler()  # FP16
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        max_lr=lr,
        epochs=max_epochs,
        optimizer=optimizer,
        steps_per_epoch=int(len(train_dataset) / batch_size),
        pct_start=0.1,
        div_factor=10,
        final_div_factor=100,
        base_momentum=0.90,
        max_momentum=0.95,
        )
    
    for epoch in range(max_epochs):
        train(epoch, model, optimizer, criterion, scaler, scheduler)
        valid(epoch, model, optimizer, criterion)

main()